In [ ]:
import pickle as pkl
import time
import os
import numpy as np
import sys
import mido
from mido import MidiFile ,MetaMessage, Message, MidiTrack, second2tick
from models.LSTM_SELFTRY import create_network, sample_with_temp

import matplotlib.pyplot as plt


# parameters

In [ ]:
# run params
section = 'compose'
run_id = '1117'
music_name = 'midis'
run_folder = 'run/{}/'.format(section)
run_folder += '_'.join([run_id, music_name])

# model params
embed_size = 200
rnn_units = 256
use_attention = True


## load the lookup tables

In [ ]:

store_folder = os.path.join(run_folder, 'store')

with open(os.path.join(store_folder, 'distincts'), 'rb') as filepath:
    distincts = pkl.load(filepath)
    command_names, n_commands, channel_names, n_channel, value1_names, n_value1, value2_names, n_value2, duration_names, n_durations = distincts

with open(os.path.join(store_folder, 'lookups'), 'rb') as filepath:
    lookups = pkl.load(filepath)
    command_to_int, int_to_command, channel_to_int, int_to_channel, value1_to_int, int_to_value1, value2_to_int, int_to_value2, duration_to_int, int_to_duration = lookups

## build the model

In [ ]:
weights_folder = os.path.join(run_folder, 'weights')
weights_file = 'weights_custom.h5'
#weights_file = 'weights-improvement-138-5.0825-bigger_custom.h5'
#weights_file = 'weights_14h_trained.h5'

model, att_model = create_network(n_commands, n_channel, n_value1, n_value2, n_durations, embed_size, rnn_units, use_attention)

# Load the weights to each node
weight_source = os.path.join(weights_folder,weights_file)
model.load_weights(weight_source)
model.summary()

## build your own phrase

In [ ]:
# prediction params
commands_temp = 0.5
channel_temp = 0.5
values1_temp = 0.5
values2_temp = 0.5
duration_temp = 0.5

max_extra_commands = 50
max_seq_len = 128
sequence_length = 64

commands = ['START']
channels = [-1]
values1 = [-1]
values2 = [-1]
durations = [0]

## Generate notes from the neural network based on a sequence of notes

In [ ]:
def generateMidoMessage(command, channel, value1, value2, duration):
    if command != 'time_signature' and command != 'key_signature':
        value1 = int(value1)
        value2 = int(value2)
        
    duration = int(duration)
    pChannel = int(channel)
    
    if command == 'set_tempo':
        return MetaMessage(command, tempo=value1, time=duration)
    elif command == 'time_signature':
        
        splitted = value1.split('/')
        pNumerator = int(splitted[0])
        pDenominator = int(splitted[1])
        
        splitted2 = value2.split('/')
        pClocks_per_click = int(splitted2[0])
        pNotated_32nd_notes_per_beat = int(splitted2[1])
        
        return MetaMessage(command, numerator=pNumerator, denominator=pDenominator, clocks_per_click=pClocks_per_click, notated_32nd_notes_per_beat=pNotated_32nd_notes_per_beat, time=duration)
    elif command == 'key_signature':    
        return MetaMessage(command, key=value1, time=duration)
    elif command == 'end_of_track':
        return MetaMessage(command, time=duration)
    elif command == 'note_on' or command == 'note_off':
        return Message(command, channel=pChannel, note=value1, velocity=value2, time=duration)
    elif command == 'control_change':
        return Message(command, channel=pChannel, control=value1,  value=value2, time=duration)
    elif command == 'program_change':
        return Message(command, channel=pChannel, program=value1, time=duration)
    elif command == 'pitchwheel':
        return Message(command, channel=pChannel, pitch=value1, time=duration)

In [ ]:
from keras.backend.tensorflow_backend import set_session
import tensorflow as tf

prediction_output = []

commands_input_sequence = []
channel_input_sequence = []
value1_input_sequence = []
value2_input_sequence = []
durations_input_sequence = []

overall_preds = []

for co, ch, v1, v2, d in zip(commands, channels, values1, values2, durations):
    command_int = command_to_int[co]
    channel_int = channel_to_int[ch]
    value1_int = value1_to_int[str(v1)]
    value2_int = value2_to_int[str(v2)]
    duration_int = duration_to_int[d]
    
    commands_input_sequence.append(command_int)
    channel_input_sequence.append(channel_int)
    value1_input_sequence.append(value1_int)
    value2_input_sequence.append(value2_int)
    durations_input_sequence.append(duration_int)
    
    prediction_output.append([co, ch, v1, v2, d])
    
    if co == 'START':
        createdMidi = MidiFile()
        track = MidiTrack()
        createdMidi.tracks.append(track)
    else:
        midi_note = note.Note(n)

        new_note = np.zeros(128)
        new_note[midi_note.pitch.midi] = 1
        overall_preds.append(new_note)


att_matrix = np.zeros(shape = (max_extra_commands+sequence_length, max_extra_commands))

for command_index in range(max_extra_commands):

    print(command_index)
    prediction_input = [
        np.asarray([commands_input_sequence])
        , np.asarray([channel_input_sequence])
        , np.asarray([value1_input_sequence])
        , np.asarray([value2_input_sequence])
        , np.asarray([durations_input_sequence])
       ]
    command_prediction, channel_prediction, value1_prediction, value2_prediction, durations_prediction = model.predict(prediction_input, verbose=0)
    if use_attention:
        att_prediction = att_model.predict(prediction_input, verbose=0)[0]
        att_matrix[(command_index-len(att_prediction)+sequence_length):(command_index+sequence_length), command_index] = att_prediction
            
    i1 = sample_with_temp(command_prediction[0], commands_temp)
    i2 = sample_with_temp(channel_prediction[0], channel_temp)
    i3 = sample_with_temp(value1_prediction[0], values1_temp)
    i4 = sample_with_temp(value2_prediction[0], values2_temp)
    i5 = sample_with_temp(durations_prediction[0], duration_temp)    

    command_result = int_to_command[i1]
    channel_result = int_to_channel[i2]
    value1_result = int_to_value1[i3]
    value2_result = int_to_value2[i4]
    duration_result = int_to_duration[i5]
    
    print(command_result, channel_result, value1_result, value2_result, duration_result)
    
    prediction_output.append([command_result, channel_result, value1_result, value2_result, duration_result])

    commands_input_sequence.append(i1)
    channel_input_sequence.append(i2)
    value1_input_sequence.append(i3)
    value2_input_sequence.append(i4)
    durations_input_sequence.append(i5)
    
    if len(commands_input_sequence) > max_seq_len:
        commands_input_sequence = commands_input_sequence[1:]
        channel_input_sequence = channel_input_sequence[1:]
        value1_input_sequence = value1_input_sequence[1:]
        value2_input_sequence = value2_input_sequence[1:]
        durations_input_sequence = durations_input_sequence[1:]
        
    if command_result == 'START':
        break
        
print('Generated sequence of {} notes'.format(len(prediction_output)))

In [ ]:
prediction_output

In [ ]:
output_folder = os.path.join(run_folder, 'output')

# create note and chord objects based on the values generated by the model
print(prediction_output)
for pattern in prediction_output:
    command, pChannel, value1, value2, duration = pattern
    pTime = duration_pattern
      
    if command == 'time_signature' or command == 'FILE' or command == 'key_signature' or command == 'sysex':
        value1 = value1
        value2 = value2
    else:
        value1 = int(value1)
        value2 = int(value2)
    
    if command == 'START':
        readMidi = MidiFile()
        track = MidiTrack()
        readMidi.tracks.append(track)
        continue
    elif command == 'sysex':
        formatedData = value1[1:len(value1)-1].split(', ')
        results = list(map(int, formatedData))
        # Quelle: https://stackoverflow.com/a/7368801
        #print('sysex message added: ' + str(value1))
        track.append(Message(command, data=results, time=duration))
        continue
    elif command == 'set_tempo':
        track.append(MetaMessage(command, tempo=value1, time=duration))
        continue
    elif command == 'time_signature':
        
        splitted = value1.split('/')
        pNumerator = int(splitted[0])
        pDenominator = int(splitted[1])
        
        splitted2 = value2.split('/')
        pClocks_per_click = int(splitted2[0])
        pNotated_32nd_notes_per_beat = int(splitted2[1])
        ### Delete ######
        #metaMessage = MetaMessage(command, numerator=pNumerator, denominator=pDenominator, clocks_per_click=pClocks_per_click, notated_32nd_notes_per_beat=pNotated_32nd_notes_per_beat, time=duration)
        #print(metaMessage)
        #################
        track.append(MetaMessage(command, numerator=pNumerator, denominator=pDenominator, clocks_per_click=pClocks_per_click, notated_32nd_notes_per_beat=pNotated_32nd_notes_per_beat, time=duration))
        continue
    elif command == 'key_signature':    
        track.append(MetaMessage(command, key=value1, time=duration))
        continue
    elif command == 'end_of_track':
        track.append(MetaMessage(command, time=duration))
        continue
    elif command == 'note_on' or command == 'note_off':
        if value1 < 0 or value2 < 0 or pChannel < 0:
            continue
        track.append(Message(command, channel=pChannel, note=value1, velocity=value2, time=duration))
        continue
    elif command == 'control_change':
        if value1 < 0 or value2 < 0 or pChannel < 0:
            continue
        track.append(Message(command, channel=pChannel, control=value1,  value=value2, time=duration))
        continue
    elif command == 'program_change':
        track.append(Message(command, channel=pChannel, program=value1, time=duration))
        continue
    elif command == 'pitchwheel':
        track.append(Message(command, channel=pChannel, pitch=value1, time=duration))
        continue
        
        
        
timestr = time.strftime("%Y%m%d-%H%M%S")
readMidi.save(os.path.join(output_folder, 'output-' + timestr + '.mid'))